Deep Learning
=============

Assignment 3
------------

Previously in `2_fullyconnected.ipynb`, you trained a logistic regression and a neural network model.

The goal of this assignment is to explore regularization techniques.

In [2]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle

First reload the data we generated in _notmist.ipynb_.

In [3]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (190000, 28, 28) (190000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [4]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 2 to [0.0, 1.0, 0.0 ...], 3 to [0.0, 0.0, 1.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (190000, 784) (190000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


In [5]:
def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

---
Problem 1
---------

Introduce and tune L2 regularization for both logistic and neural network models. Remember that L2 amounts to adding a penalty on the norm of the weights to the loss. In TensorFlow, you can compute the L2 loss for a tensor `t` using `nn.l2_loss(t)`. The right amount of regularization should improve your validation / test accuracy.

---

In [30]:
batch_size = 128
hidden_nodes_layer_1 = 1024
learning_rate = .2

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_regularization_ammount = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_layer_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_layer_1]))
  biases_layer_1 = tf.Variable(tf.zeros([hidden_nodes_layer_1]))
    
  
  weights_layer_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_layer_1, num_labels]))
  biases_layer_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  ouput_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_layer_1)) + biases_layer_1
  logits = tf.matmul(ouput_1,weights_layer_2)+biases_layer_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
            logits, tf_train_labels))+ tf_regularization_ammount*tf.nn.l2_loss(weights_layer_1)+ tf_regularization_ammount*tf.nn.l2_loss(weights_layer_2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_layer_1) + biases_layer_1),weights_layer_2)+biases_layer_2)

  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_layer_1) + biases_layer_1),weights_layer_2)+biases_layer_2)

In [31]:
num_steps = 1001
regularization_amount = 0.002

with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    indexes = np.random.choice(train_labels.shape[0], batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[indexes, :]
    batch_labels = train_labels[indexes, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_regularization_ammount : regularization_amount}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 927.100586
Minibatch accuracy: 4.7%
Validation accuracy: 38.7%
Minibatch loss at step 100: 605.098694
Minibatch accuracy: 78.9%
Validation accuracy: 79.5%
Minibatch loss at step 200: 576.338562
Minibatch accuracy: 71.9%
Validation accuracy: 74.7%
Minibatch loss at step 300: 511.916321
Minibatch accuracy: 77.3%
Validation accuracy: 71.7%
Minibatch loss at step 400: 479.088013
Minibatch accuracy: 77.3%
Validation accuracy: 80.1%
Minibatch loss at step 500: 431.654388
Minibatch accuracy: 84.4%
Validation accuracy: 82.4%
Minibatch loss at step 600: 399.560272
Minibatch accuracy: 78.9%
Validation accuracy: 81.7%
Minibatch loss at step 700: 360.971191
Minibatch accuracy: 85.2%
Validation accuracy: 81.2%
Minibatch loss at step 800: 336.139618
Minibatch accuracy: 81.2%
Validation accuracy: 78.3%
Minibatch loss at step 900: 309.565613
Minibatch accuracy: 76.6%
Validation accuracy: 75.3%
Minibatch loss at step 1000: 281.217407
Minibatch accuracy: 86.7%
Valid

---
Problem 2
---------
Let's demonstrate an extreme case of overfitting. Restrict your training data to just a few batches. What happens?

---

* If we keep the same ammount of regularization (.002) test performance decreases to ~77%.
* If we increase the ammount of regularization (.1) performance seems to be better ~84%

 * We also see that training data accuracy reach 100% with small regularization; however validation accuracy keeps smaller. The NN is overfitting.

With greater regularization value training and validation accuracy tend to be more similar.

In [34]:
batch_size = 128
hidden_nodes_layer_1 = 1024
learning_rate = .2
num_steps = 1001
regularization_amount = 0.1

size_truncated_dataset = 1000
indexes = np.random.choice(train_labels.shape[0],size_truncated_dataset)
train_dataset_truncated = train_dataset[indexes, :]
train_labels_truncated = train_labels[indexes, :]
print('Training set truncated:', train_dataset_truncated.shape, train_labels_truncated.shape)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_regularization_ammount = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_layer_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_layer_1]))
  biases_layer_1 = tf.Variable(tf.zeros([hidden_nodes_layer_1]))
    
  
  weights_layer_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_layer_1, num_labels]))
  biases_layer_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  ouput_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_layer_1)) + biases_layer_1
  logits = tf.matmul(ouput_1,weights_layer_2)+biases_layer_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
            logits, tf_train_labels))+ tf_regularization_ammount*tf.nn.l2_loss(weights_layer_1)+ tf_regularization_ammount*tf.nn.l2_loss(weights_layer_2)
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_layer_1) + biases_layer_1),weights_layer_2)+biases_layer_2)

  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_layer_1) + biases_layer_1),weights_layer_2)+biases_layer_2)



with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    indexes = np.random.choice(train_labels_truncated.shape[0], batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_truncated[indexes, :]
    batch_labels = train_labels_truncated[indexes, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_regularization_ammount : regularization_amount}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Training set truncated: (1000, 784) (1000, 10)
Initialized
Minibatch loss at step 0: 31805.929688
Minibatch accuracy: 10.2%
Validation accuracy: 33.0%
Minibatch loss at step 100: 553.147461
Minibatch accuracy: 99.2%
Validation accuracy: 77.0%
Minibatch loss at step 200: 11.012300
Minibatch accuracy: 91.4%
Validation accuracy: 78.1%
Minibatch loss at step 300: 1.620971
Minibatch accuracy: 79.7%
Validation accuracy: 77.9%
Minibatch loss at step 400: 1.368852
Minibatch accuracy: 82.8%
Validation accuracy: 78.3%
Minibatch loss at step 500: 1.355746
Minibatch accuracy: 82.8%
Validation accuracy: 77.5%
Minibatch loss at step 600: 1.373718
Minibatch accuracy: 80.5%
Validation accuracy: 78.0%
Minibatch loss at step 700: 1.342601
Minibatch accuracy: 83.6%
Validation accuracy: 78.5%
Minibatch loss at step 800: 1.305288
Minibatch accuracy: 84.4%
Validation accuracy: 78.6%
Minibatch loss at step 900: 1.364587
Minibatch accuracy: 82.0%
Validation accuracy: 77.2%
Minibatch loss at step 1000: 1.36454

---
Problem 3
---------
Introduce Dropout on the hidden layer of the neural network. Remember: Dropout should only be introduced during training, not evaluation, otherwise your evaluation results would be stochastic as well. TensorFlow provides `nn.dropout()` for that, but you have to make sure it's only inserted during training.

What happens to our extreme overfitting case?

---

With `keep_prob_dropout` probability ~10% we significantly improve the accuracy: (87.6%).

We also see similar validation-training error thus we conclude overfitting is not taking place.

It is also curious that validation error accuracy reaches ~ 80% around step 300: whereas training accuracy is smaller. I think this is due to that training prediction is only taking into account the 10% active hidden neurons at this step; whereas valid prediction takes into account all hidden units.

In [54]:
batch_size = 128
hidden_nodes_layer_1 = 1024
learning_rate = .02
num_steps = 1001
keep_prob_dropout = 0.1
size_truncated_dataset = 1000

indexes = np.random.choice(train_labels.shape[0],size_truncated_dataset)
train_dataset_truncated = train_dataset[indexes, :]
train_labels_truncated = train_labels[indexes, :]

print('Training set truncated:', train_dataset_truncated.shape, train_labels_truncated.shape)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_regularization_ammount = tf.placeholder(tf.float32)
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_layer_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_layer_1]))
  biases_layer_1 = tf.Variable(tf.zeros([hidden_nodes_layer_1]))
    
  
  weights_layer_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_layer_1, num_labels]))
  biases_layer_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  ouput_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_layer_1)) + biases_layer_1
  tf_keep_prob_dropout = tf.placeholder("float")
  ouput_1_drop = tf.nn.dropout(ouput_1, tf_keep_prob_dropout)
  
  logits = tf.matmul(ouput_1_drop,weights_layer_2)+biases_layer_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
            logits, tf_train_labels))
  
  # Optimizer.
  optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_layer_1) + biases_layer_1),weights_layer_2)+biases_layer_2)

  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_layer_1) + biases_layer_1),weights_layer_2)+biases_layer_2)


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    indexes = np.random.choice(train_labels_truncated.shape[0], batch_size)
    # Generate a minibatch.
    batch_data = train_dataset_truncated[indexes, :]
    batch_labels = train_labels_truncated[indexes, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_keep_prob_dropout : keep_prob_dropout}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      print("Validation accuracy: %.1f%%" % accuracy(
        valid_prediction.eval(), valid_labels))
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Training set truncated: (1000, 784) (1000, 10)
Initialized
Minibatch loss at step 0: 1223.686401
Minibatch accuracy: 12.5%
Validation accuracy: 11.4%
Minibatch loss at step 100: 193.443100
Minibatch accuracy: 56.2%
Validation accuracy: 77.6%
Minibatch loss at step 200: 150.274567
Minibatch accuracy: 60.9%
Validation accuracy: 79.0%
Minibatch loss at step 300: 101.042603
Minibatch accuracy: 62.5%
Validation accuracy: 79.7%
Minibatch loss at step 400: 82.962830
Minibatch accuracy: 76.6%
Validation accuracy: 80.3%
Minibatch loss at step 500: 65.724686
Minibatch accuracy: 70.3%
Validation accuracy: 80.4%
Minibatch loss at step 600: 50.038368
Minibatch accuracy: 74.2%
Validation accuracy: 80.4%
Minibatch loss at step 700: 42.697342
Minibatch accuracy: 77.3%
Validation accuracy: 80.6%
Minibatch loss at step 800: 33.027416
Minibatch accuracy: 75.8%
Validation accuracy: 80.8%
Minibatch loss at step 900: 44.344795
Minibatch accuracy: 79.7%
Validation accuracy: 80.7%
Minibatch loss at step 1000:

---
Problem 4
---------

Try to get the best performance you can using a multi-layer model! The best reported test accuracy using a deep network is [97.1%](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html?showComment=1391023266211#c8758720086795711595).

One avenue you can explore is to add multiple layers.

Another one is to use learning rate decay:

    global_step = tf.Variable(0)  # count the number of steps taken.
    learning_rate = tf.train.exponential_decay(0.5, global_step, ...)
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
 
 ---


In [23]:
batch_size = 128
hidden_nodes_layer_1 = 1024
print('Training set :', train_dataset.shape, train_labels.shape)
graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(batch_size, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  weights_layer_1 = tf.Variable(
    tf.truncated_normal([image_size * image_size, hidden_nodes_layer_1]))
  biases_layer_1 = tf.Variable(tf.zeros([hidden_nodes_layer_1]))
    
  
  weights_layer_2 = tf.Variable(
    tf.truncated_normal([hidden_nodes_layer_1, num_labels]))
  biases_layer_2 = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  ouput_1 = tf.nn.relu(tf.matmul(tf_train_dataset, weights_layer_1)) + biases_layer_1
  tf_keep_prob_dropout = tf.placeholder("float")
  ouput_1_drop = tf.nn.dropout(ouput_1, tf_keep_prob_dropout)

  logits = tf.matmul(ouput_1_drop,weights_layer_2)+biases_layer_2
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(
            logits, tf_train_labels))
  
  # Optimizer.
  tf_learning_rate = tf.placeholder("float")
  optimizer = tf.train.GradientDescentOptimizer(tf_learning_rate).minimize(loss)
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_valid_dataset, weights_layer_1) + biases_layer_1),weights_layer_2)+biases_layer_2)

  test_prediction = tf.nn.softmax(
    tf.matmul(tf.nn.relu(tf.matmul(tf_test_dataset, weights_layer_1) + biases_layer_1),weights_layer_2)+biases_layer_2)


Training set : (190000, 784) (190000, 10)


In [24]:
num_steps = 3001
keep_prob_dropout = 0.55
learning_rate = .5
accuracy_validation_before = 0


with tf.Session(graph=graph) as session:
  tf.initialize_all_variables().run()
  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    indexes = np.random.choice(train_labels.shape[0], batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[indexes, :]
    batch_labels = train_labels[indexes, :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels,tf_keep_prob_dropout : keep_prob_dropout,tf_learning_rate: learning_rate}
    _, l, predictions = session.run(
      [optimizer, loss, train_prediction], feed_dict=feed_dict)
    if (step % 100 == 0):
      print("Minibatch loss at step %d: %f" % (step, l))
      current_accuracy = accuracy(
        valid_prediction.eval(), valid_labels)
      if (current_accuracy < accuracy_validation_before):
        learning_rate*=.5
        print("Reducing learning rate to: %.4f%%" %learning_rate)
      print("Minibatch accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      accuracy_validation_before = current_accuracy
      print("Validation accuracy: %.1f%%" % current_accuracy)
  print("Test accuracy: %.1f%%" % accuracy(test_prediction.eval(), test_labels))

Initialized
Minibatch loss at step 0: 410.861298
Minibatch accuracy: 12.5%
Validation accuracy: 29.4%
Minibatch loss at step 100: 72.365723
Minibatch accuracy: 69.5%
Validation accuracy: 80.5%
Minibatch loss at step 200: 52.781815
Reducing learning rate to: 0.2500%
Minibatch accuracy: 76.6%
Validation accuracy: 74.5%
Minibatch loss at step 300: 26.808149
Minibatch accuracy: 76.6%
Validation accuracy: 82.0%
Minibatch loss at step 400: 23.338087
Minibatch accuracy: 81.2%
Validation accuracy: 82.5%
Minibatch loss at step 500: 20.922606
Minibatch accuracy: 76.6%
Validation accuracy: 83.2%
Minibatch loss at step 600: 19.986942
Reducing learning rate to: 0.1250%
Minibatch accuracy: 77.3%
Validation accuracy: 83.0%
Minibatch loss at step 700: 16.960430
Minibatch accuracy: 68.8%
Validation accuracy: 84.3%
Minibatch loss at step 800: 10.646368
Reducing learning rate to: 0.0625%
Minibatch accuracy: 82.8%
Validation accuracy: 84.2%
Minibatch loss at step 900: 19.262796
Minibatch accuracy: 75.0%
V